In [9]:
import pandas as pd 
import numpy as np
select_blackbox = ['dt', 'nn', 'rf'][1]
sanity = False
if(sanity):
    df = pd.read_csv("data/output/sanity_result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']

else:
    df = pd.read_csv("data/output/result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
    df = df[df['blackbox'] == select_blackbox]


print("minimum iterations:", min(df.groupby(['query'])['explanation'].count()), "max iterations:",max(df.groupby(['query'])['explanation'].count()))
df.head()

KeyError: 'blackbox'

In [3]:
# take median
group_list = ['dataset','explainer','query', 'syntactic grammar']
df_med = df.groupby(group_list).median()
df_med.reset_index(inplace=True)
df_med

,dataset,explainer,query,syntactic grammar,accuracy,explanation size,positive counterexamples,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,zoo,sygus,breathes = 1,0.0,1.000000,1.0,0.000000,89.0,1.0,0.051779,0.028197,0.023548,15.5
1,zoo,sygus,breathes = 1,1.0,1.000000,1.0,0.000000,89.0,1.0,0.065364,0.041451,0.022833,5.5
2,zoo,sygus,breathes >= 0,0.0,0.818182,1650.5,0.915538,209194.0,0.0,101.058080,73.555758,26.444369,184.0
3,zoo,sygus,breathes >= 0,1.0,1.000000,2.0,0.267857,165.5,1.0,0.542820,0.393466,0.145502,10.5
4,zoo,sygus,fins = 0,0.0,1.000000,1.0,0.000000,89.0,1.0,0.064683,0.033741,0.030472,15.5
5,zoo,sygus,fins = 0,1.0,1.000000,1.0,0.000000,89.0,1.0,0.084493,0.053889,0.029768,5.5
6,zoo,sygus,milk = 1,0.0,1.000000,1566.0,0.919271,231451.5,0.0,101.058845,74.822900,25.182061,192.0
7,zoo,sygus,milk = 1,1.0,1.000000,3.0,0.279221,373.5,1.0,1.371359,1.155108,0.209128,13.5


In [4]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['dataset','explainer','query', 'syntactic grammar'], as_index = False):
    item.reset_index(inplace=True, drop=True)
    if(len(item) == 1):
        query_explanations[key] = item['explanation'].item()
    else:
        # nearest explanation to the average accuracy
        explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['dataset'] == key[0]) & (df_med['explainer'] == key[1]) & (df_med['syntactic grammar'] == key[3]) & (df_med['query'] == key[2])].item()).abs().argsort()][0]]['explanation']
        query_explanations[key] = explanation

In [5]:
df_output = df_med
df_output = df_output.drop("terminate", axis=1)
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][(df_output['dataset'] == key[0]) & (df_output['explainer'] == key[1]) & (df_output['syntactic grammar'] == key[3]) & (df_output['query'] == key[2])] = query_explanations[key]

# reorganise columns
df_output = df_output[[ 'dataset',  'syntactic grammar', 'explainer','query', 'explanation', 'accuracy', 'time', 'time learner', 'time verifier', 'random words checked', 'total counterexamples', 'positive counterexamples']]
df_output = df_output.round(2)
df_output.head(n=10)

,dataset,syntactic grammar,explainer,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples
0,zoo,0.0,sygus,breathes = 1,false,1.00,0.05,0.03,0.02,89.0,15.5,0.00
1,zoo,1.0,sygus,breathes = 1,false,1.00,0.07,0.04,0.02,89.0,5.5,0.00
2,zoo,0.0,sygus,breathes >= 0,(let ((_let_0 (= legs (/ 467 500)))) (let ((_...,0.82,101.06,73.56,26.44,209194.0,184.0,0.92
3,zoo,1.0,sygus,breathes >= 0,(and (not breathes) fins),1.00,0.54,0.39,0.15,165.5,10.5,0.27
4,zoo,0.0,sygus,fins = 0,false,1.00,0.06,0.03,0.03,89.0,15.5,0.00
5,zoo,1.0,sygus,fins = 0,false,1.00,0.08,0.05,0.03,89.0,5.5,0.00
6,zoo,0.0,sygus,milk = 1,(let ((_let_0 (= legs (/ 113 200)))) (let ((_...,1.00,101.06,74.82,25.18,231451.5,192.0,0.92
7,zoo,1.0,sygus,milk = 1,(and milk (and (not breathes) fins)),1.00,1.37,1.16,0.21,373.5,13.5,0.28


In [7]:
import matplotlib.pyplot as plt
import pickle
from IPython.display import Markdown, display
from trustable_explanation import helper_functions
import numpy as np


features = ['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize']
# print results in an explainable manner
for key, item in df_output.groupby(['dataset','query','explainer','syntactic grammar'], as_index = False):
    print("\n\n")
    display(Markdown("### Query"))
    for i in key:
        print(i)
    if(key[3] == 0):
        continue
    display(Markdown("### Result"))
    if(key[1] == "logistic regression"):
        feature_importance = np.fromstring(item['explanation'].item()[1:-1], dtype=np.float, sep=' ')
        feature_importance = 100.0 * (feature_importance / (abs(feature_importance).max()))
        sorted_idx = np.argsort(abs(feature_importance))
        pos = np.arange(sorted_idx.shape[0]) + .5
        featfig = plt.figure()
        featax = featfig.add_subplot(1, 1, 1)
        featax.barh(pos, feature_importance[sorted_idx], align='center')
        featax.set_yticks(pos)
        featax.set_yticklabels(np.array(features)[sorted_idx])
        featax.set_xlabel('Relative Feature Importance')
        plt.tight_layout()   
        plt.show()
    elif(key[1] == "decision tree"):
        dt = None
        with open(item['explanation'].item(), 'rb') as fid:
            dt = pickle.load(fid)
        print(helper_functions.tree_to_code(dt,features))
    else:
        print(item['explanation'].item())
    print("\n\naccuracy",item['accuracy'].item())


### Query

zoo
breathes = 1
sygus
0.0





### Query

zoo
breathes = 1
sygus
1.0


### Result

...

zoo
milk = 1
sygus
0.0





### Query

zoo
milk = 1
sygus
1.0


### Result

(and milk (and (not breathes) fins))


accuracy 1.0


In [70]:
# to remote 
# df_output.to_csv("data/output/summary.csv", index = False)
df_output.to_csv("/home/bishwamittra/Dropbox/trustable_explanations/result.csv", index = False)